In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/kaggle/QIA2023"

/content/drive/MyDrive/kaggle/QIA2023


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset


from transformers import *
import os
import sys
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import numpy as np
import re
import pickle
import time
import pandas as pd
from pathlib import Path
import random
from torch.utils.tensorboard import SummaryWriter

/usr/local/lib/python3.10/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation_flax_utils.py:24: FutureWarning: Importing `FlaxGenerationMixin` from `src/transformers/generation_flax_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import FlaxGenerationMixin` instead.
  warnings.warn(
Xformers is not installed correctly. If you want to use memorry_efficient_attenti

# 1. Embedding

In [ ]:
# load datasets
df = pd.read_csv('./data/train_data.csv', index_col=0)


In [ ]:
df.head()

,User_ID,Gender,Age,MBTI,Q_number,Short_Answer,Long_Answer
Data_ID,,,,,,,
481,9,female,30,INTJ,1,아니다,새로운 사람을 만나서 이야기를 나누고 마음이 통하는 친구가 되기까지의 과정은 많은 ...
482,9,female,30,INTJ,2,그렇다,자유시간이 생기면 평소에 관심을 두고 있던 분야에 대해 공부하고 탐구하는 시간을 주...
483,9,female,30,INTJ,3,그렇다,다른 사람이 울고 있는 모습만 볼 때는 왜 울고 있는지 이유를 몰라 공감 되지 않는...
484,9,female,30,INTJ,4,그렇다,모든 일이 마음먹은 대로 진행되는 것은 아니기에 미리 여러 가지 대책을 세워둬야 계...
485,9,female,30,INTJ,5,그렇다,퇴사 면담을 하면서 대외적인 이유가 아닌 진짜 퇴사의 이유를 솔직하게 말해 달라는 ...


In [ ]:
question = pd.read_csv('./data/question.csv')

In [ ]:
question = question.dropna().drop('index.1',axis=1)

In [ ]:
question['index']=question['index'].astype('int')
question

In [ ]:
df['Q&A']=df.apply(lambda row:question.iloc[row['Q_number']-1]['Question']+' '+row['Short_Answer'],axis=1)

df['Q&A']

In [ ]:
df

In [ ]:
# load pretrained model
def get_model():
    # * Model          | Tokenizer          | Pretrained weights shortcut
    # MODEL=(DistilBertModel, DistilBertTokenizer, 'distilbert-base-uncased')
    tokenizer = BertTokenizerFast.from_pretrained("klue/bert-base")
    model = BertModel.from_pretrained("klue/bert-base")
    n_hl = model.config.num_hidden_layers
    return model, tokenizer, n_hl

In [ ]:
model, tokenizer, n_hl = get_model()

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/tokenizer_config.json


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/config.json
Model config BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/config.json
Model config BertConfig {
  "architectu

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--klue--bert-base/snapshots/34b965303f98bc5214daca7f76b7fb82d2dc6183/pytorch_model.bin
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification mo

In [ ]:
df['Gender'] = df['Gender'].map({'female':1,'male':0})

In [ ]:
# split train and test dataframe
train_df_list = []
test_df_list = []
for idx in df['User_ID'].unique():
    train_df_list.append(df[df['User_ID']==idx][0:50])
    test_df_list.append(df[df['User_ID']==idx][50:])

In [ ]:
train_df = pd.concat(train_df_list, ignore_index=True)
test_df = pd.concat(test_df_list, ignore_index=True)

In [ ]:
train_df

,User_ID,Gender,Age,MBTI,Q_number,Short_Answer,Long_Answer
0,9,1,30,INTJ,1,아니다,새로운 사람을 만나서 이야기를 나누고 마음이 통하는 친구가 되기까지의 과정은 많은 ...
1,9,1,30,INTJ,2,그렇다,자유시간이 생기면 평소에 관심을 두고 있던 분야에 대해 공부하고 탐구하는 시간을 주...
2,9,1,30,INTJ,3,그렇다,다른 사람이 울고 있는 모습만 볼 때는 왜 울고 있는지 이유를 몰라 공감 되지 않는...
3,9,1,30,INTJ,4,그렇다,모든 일이 마음먹은 대로 진행되는 것은 아니기에 미리 여러 가지 대책을 세워둬야 계...
4,9,1,30,INTJ,5,그렇다,퇴사 면담을 하면서 대외적인 이유가 아닌 진짜 퇴사의 이유를 솔직하게 말해 달라는 ...
...,...,...,...,...,...,...,...
5995,240,1,30,ISTP,46,중립/모르겠다,현재의 삶에 제 가족과 딸을 보고 내가 살아가는 이유를 생각하고 살아가고 있습니다
5996,240,1,30,ISTP,47,아니다,한번 감정이 휘둘리면 못 말리는 성격이라 감정 자제 좀 하라고 말을 많이 듣고 있습니다
5997,240,1,30,ISTP,48,그렇다,상대방의 체면을 살려주기 위해 다른 거에 빚대 어서 말을 해주곤 합니다
5998,240,1,30,ISTP,49,그렇다,일이 있으면 끝까지 처리해야지 직성이 풀려서 끝까지 일을 하고 마무리를 합니다 회사...


In [ ]:
deliminator = pd.Series([' ' for i in range(len(train_df['Long_Answer']))])

train_df['Answer_added'] = train_df['Long_Answer'].add(deliminator) 
train_df['Answer_added'] = train_df['Answer_added'].add(train_df['Age'].astype('str')) 
train_df['Answer_added'] = train_df['Answer_added'].add(deliminator) 
train_df['Answer_added'] = train_df['Answer_added'].add(train_df['Gender'].astype('str'))
train_df['Answer_added'][0]

deliminator = pd.Series([' ' for i in range(len(test_df['Long_Answer']))])
test_df['Answer_added'] = test_df['Long_Answer'].add(deliminator) 
test_df['Answer_added'] = test_df['Answer_added'].add(test_df['Age'].astype('str')) 
test_df['Answer_added'] = test_df['Answer_added'].add(deliminator) 
test_df['Answer_added'] = test_df['Answer_added'].add(test_df['Gender'].astype('str'))
test_df['Answer_added'][0]

'일을 수행하는 데 온전히 집중하고 싶습니다. 많은 사람과 함께 일하다 보면 사사롭고 불필요한 일에 신경 써야 할 때가 많은데 시간 낭비라고 생각합니다. 30 1'

In [ ]:
train_tensor = tokenizer(train_df['Answer_added'].to_list(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)
test_tensor = tokenizer(test_df['Answer_added'].to_list(), max_length=model.config.max_position_embeddings, return_tensors='pt', padding=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2382: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [ ]:
class MyMapDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data['input_ids'])

    def __getitem__(self, idx):
        data = {k:v[idx] for k,v in self.data.items()}
        return data

In [ ]:
train_ds = MyMapDataset(train_tensor)
test_ds = MyMapDataset(test_tensor)

train_dl = DataLoader(train_ds, batch_size=32, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=False)

In [ ]:
def forward(model, dl, device=0):
    pooled = []
    hidden = []
    model.cuda(device)
    model.eval()
    for data in dl:
        data = {k:v.cuda(device) for k,v in data.items()}
        with torch.no_grad():
            output = model(**data, output_hidden_states=True)
        p, h = output.pooler_output, output.hidden_states
        pooled.append(p) # pooler output
        hidden.append(h[-1][:,0,:]) # only [CLS] token embedding 
    return torch.cat(pooled), torch.cat(hidden)

In [ ]:
train_result = forward(model, train_dl, device=0)
test_result = forward(model, test_dl, device=0)

In [ ]:
torch.save(train_result, 'train_embed_only_p2.pt')
torch.save(test_result, 'test_embed_only_p2.pt')